In [2]:
import pandas as pd
from pickle import load
from numpy import array
import numpy as np
import pydot
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras import optimizers
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.layers import Dense
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.layers import Activation
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
import random
from pyts.image import GramianAngularField, RecurrencePlot, MarkovTransitionField
import matplotlib.pyplot as plt

pref = 'traintest/1000_'

Using TensorFlow backend.


In [5]:

X_train = np.load(pref+'x_train.npy')
X_val = np.load(pref+'x_val.npy')
X_test = np.load(pref+'x_test.npy')
y_train = np.load(pref+'y_train.npy')
y_val = np.load(pref+'y_val.npy')
y_test = np.load(pref+'y_test.npy')


In [7]:
image_size = 1.
def to_images(data, batch):
    gasf = GramianAngularField(image_size)
    cont = True
    low = 0
    high = 100
    gasf.fit(data)
    X_gasf = np.zeros((len(data),len(data[0]), len(data[0])))
    while cont == True:
        X_gasf[low:high] = gasf.transform(data[low:high])   
        low = high
        high += batch
        if high > len(data):
            cont = False
        print(high)
    X_gasf[low:len(X_gasf)] = gasf.transform(data[low:len(data)])    # last bunch of records
    return X_gasf

lim = 1000
im_train = to_images(X_train[0:lim], 120)

lim = 150
im_val = to_images(X_val[0:lim], 120)
im_test = to_images(X_test[0:lim], 120)


220
340
460
580
700
820
940
1060
220
220


In [10]:
np.save(pref+'im_train',im_train)
np.save(pref+'im_val',im_val)
np.save(pref+'im_test',im_test)



In [4]:
im_train = np.load(pref+'im_train.npy')
im_val = np.load(pref+'im_val.npy')
im_test = np.load(pref+'im_test.npy')

#for item in X_gasf[0:10]
#    plt.imshow(item, cmap='rainbow', origin='lower')
#    plt.title('GASF', fontsize=16)
#    plt.show()


In [7]:
im_train = im_train.reshape(1000, 1000, 1000,1)
im_val = im_val.reshape(150, 1000, 1000,1)
im_test = im_test.reshape(150, 1000, 1000,1)
y_train = np.load(pref+'y_train.npy')
y_val = np.load(pref+'y_val.npy')
y_test = np.load(pref+'y_test.npy')

In [1]:
def larger_model():
    # create model
    model = Sequential()
    model.add(Conv2D(30, (100, 100), input_shape=(1000, 1000, 1), activation='relu'))
#    model.add(MaxPooling2D(pool_size=(2, 2)))
#    model.add(Conv2D(15, (3, 3), activation='relu'))
#    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
#    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(y_train.shape[1],activation='sigmoid'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
#X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
#X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')

model = larger_model()
model.fit(im_train[:100], y_train[:100], validation_data=(im_val[:10], y_val[:10]), epochs=2, batch_size=200, verbose=2)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))


NameError: name 'Sequential' is not defined